In [6]:
import numpy as np

import torch
from torch.utils.data import DataLoader, RandomSampler
from scripts.pums_downloader import datasets

from scripts.pums_sgd import PumsModule, load_pums, problem, evaluate, printf

In [95]:
batch_size = 10000
train_loader = DataLoader(load_pums(datasets[1]), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(load_pums(datasets[1]), batch_size=batch_size)
# train_loader.sampler = RandomSampler(replacement=True)

In [3]:
model_ = PumsModule(len(problem['predictors']), 2)
gradients = dict((name, []) for name, _ in model_.named_parameters())

printf(f"acc\t|\tloss")
for batch in train_loader:
    model = PumsModule(len(problem['predictors']), 2)
    optimizer = torch.optim.SGD(model.parameters(), .1)
    loss = model.loss(batch)
    loss.backward()

    # gradients.append(list(model.linear1.parameters()))
    for name, param in model.named_parameters():
        if param.requires_grad:
            grad = param.grad.detach().clone()
            gradients[name].append(grad)
    optimizer.step()
    # print(model.linear1.weight.grad)
    # print(model.linear1.bias.grad)

    optimizer.zero_grad()

    accuracy, loss = evaluate(model, test_loader)
    printf(f"{accuracy.item():.2f}\t|\t{loss.item():.2f}", force=True)

acc	|	loss
0.93	|	135.83
0.93	|	0.58
0.93	|	5.82
0.07	|	20.66


In [7]:
# This should be a dp median
import torch.nn as nn
avg_linear1 = sum(gradients['linear1.weight'])  / len(gradients['linear1.weight'])
avg_linear1_bias = sum(gradients['linear1.bias']) / len(gradients['linear1.bias'])
avg_linear2 = sum(gradients['linear2.weight']) / len(gradients['linear2.weight'])
avg_linear2_bias = sum(gradients['linear2.bias']) / len(gradients['linear2.bias'])

model.linear1.weight = nn.Parameter(avg_linear1)
model.linear1.bias = nn.Parameter(avg_linear1_bias)
model.linear2.weight = nn.Parameter(avg_linear2)
model.linear2.bias = nn.Parameter(avg_linear2_bias)

acc, loss = evaluate(model, test_loader)
acc, loss

UsageError: Line magic function `%%` not found.


In [20]:
gradients['linear1.weight']

[tensor([[-6.9835e+00, -1.6266e+01, -3.2259e-01],
         [ 5.7672e+00,  1.2583e+01,  2.5878e-01],
         [-5.1285e-01, -2.6154e-01, -1.5506e-02],
         [ 1.4230e+01,  3.3144e+01,  6.5731e-01],
         [ 2.2056e+00,  1.5239e+00,  6.0899e-02]]),
 tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.0806e-01, 2.4868e-02, 2.8414e-03],
         [7.3711e+00, 2.0279e+01, 3.7872e-01],
         [2.4274e-01, 2.3311e-01, 7.9131e-03]]),
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.8052e-01,  9.2190e-02,  5.4301e-03],
         [ 1.5894e-01,  3.6722e-02,  4.1896e-03],
         [ 5.9543e+00,  1.4419e+01,  2.8501e-01],
         [-2.0103e+00, -2.7710e+00, -7.2354e-02]]),
 tensor([[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.5902, 0.8826, 0.0148],
         [0.6671, 0.8059, 0.0160],
         [3.4602, 5.1747, 0.0868]])]

In [27]:
example_stacked_tensor = torch.stack(gradients['linear1.weight'])
# example_stacked_tensor.slice([0,0,0])

In [28]:
[x[0][0].item() for x in gradients['linear1.weight']]

[-6.9835429191589355, 0.0, 0.0, 0.0]

In [50]:
torch.stack(gradients['linear1.weight']).reshape(4, -1)

tensor([[-6.9835e+00, -1.6266e+01, -3.2259e-01,  5.7672e+00,  1.2583e+01,
          2.5878e-01, -5.1285e-01, -2.6154e-01, -1.5506e-02,  1.4230e+01,
          3.3144e+01,  6.5731e-01,  2.2056e+00,  1.5239e+00,  6.0899e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0806e-01,  2.4868e-02,  2.8414e-03,  7.3711e+00,
          2.0279e+01,  3.7872e-01,  2.4274e-01,  2.3311e-01,  7.9131e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.8052e-01,  9.2190e-02,
          5.4301e-03,  1.5894e-01,  3.6722e-02,  4.1896e-03,  5.9543e+00,
          1.4419e+01,  2.8501e-01, -2.0103e+00, -2.7710e+00, -7.2354e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  5.9017e-01,  8.8258e-01,  1.4797e-02,  6.6711e-01,
          8.0588e-01,  1.6026e-02,  3.4602e+00,  5.1747e+00,  8.6758e-02]])

In [73]:
import numpy as np

def get_columns_for_median(name):
    for x in zip(*map(torch.flatten, gradients[name])):
        yield [y.item() for y in x]

new_model = PumsModule(len(problem['predictors']), 2)

for name, grad in gradients.items():
    columns = get_columns_for_median(name)
    medians = torch.Tensor([np.median(col) for col in columns]).resize(*gradients[name][0].shape)
    layer, param = name.split('.')
    getattr(getattr(new_model, layer), param).grad = medians

print(new_model.linear1.weight == model.linear1.weight)
print(new_model.linear1.bias == model.linear1.bias)
print(new_model.linear2.weight == model.linear2.weight)
print(new_model.linear2.bias == model.linear2.bias)

tensor([[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]])
tensor([False, False, False, False, False])
tensor([[False, False, False, False, False],
        [False, False, False, False, False]])
tensor([False, False])


/home/ecowan/IdeaProjects/smartnoise-core-python/venv/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [75]:
printf(f"acc\t|\tloss")
optimizer = torch.optim.SGD(model.parameters(), .1)
new_optimizer = torch.optim.SGD(new_model.parameters(), .1)
for batch in train_loader:
    loss = model.loss(batch)
    loss.backward()
    for name, grad in gradients.items():
        columns = get_columns_for_median(name)
        medians = torch.Tensor([np.median(col) for col in columns]).resize(*gradients[name][0].shape)
        layer, param = name.split('.')
        getattr(getattr(new_model, layer), param).grad = medians
    new_loss = new_model.loss(batch)
    new_loss.backward()
    optimizer.step()
    new_optimizer.step()
    # print(model.linear1.weight.grad)
    # print(model.linear1.bias.grad)
    optimizer.zero_grad()
    new_optimizer.zero_grad()
    accuracy, loss = evaluate(model, test_loader)
    new_acc, new_loss_val = evaluate(new_model, test_loader)
    printf(f"{accuracy.item():.2f}\t|\t{loss.item():.2f}", force=True)
    printf(f"{new_acc.item():.2f}\t|\t{new_loss_val.item():.2f}", force=True)#%%

acc	|	loss
0.93	|	0.62
0.93	|	0.49
0.93	|	0.59
0.93	|	0.54
0.93	|	0.57
0.93	|	0.46
0.93	|	0.54
0.93	|	0.40


/home/ecowan/IdeaProjects/smartnoise-core-python/venv/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [35]:
batch_size = 10000
train_loader = DataLoader(load_pums(datasets[1]), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(load_pums(datasets[1]), batch_size=batch_size)

model = PumsModule(len(problem['predictors']), 2)
gradients = dict((name, []) for name, _ in model.named_parameters())

def get_columns_for_median(name):
    for x in zip(*map(torch.flatten, gradients[name])):
        yield [y.item() for y in x]

printf(f"acc\t|\tloss")
for batch in train_loader:
    # model = PumsModule(len(problem['predictors']), 2)
    for epoch in range(0,3):
        optimizer = torch.optim.SGD(model.parameters(), .1)
        loss = model.loss(batch)
        loss.backward()

        for name, param in model.named_parameters():
            if param.requires_grad:
                grad = param.grad.detach().clone()
                gradients[name].append(grad)
        optimizer.step()
        optimizer.zero_grad()

for batch in train_loader:
    for name, grad in gradients.items():
        columns = get_columns_for_median(name)
        medians = torch.Tensor([np.median(col) for col in columns]).resize(*gradients[name][0].shape)
        # Names are of the form "linear1.weight"
        layer, param = name.split('.')
        getattr(getattr(model, layer), param).grad = medians

    optimizer.step()
    optimizer.zero_grad()

    accuracy, loss = evaluate(model, test_loader)
    printf(f"{accuracy.item():.2f}\t|\t{loss.item():.2f}", force=True)

acc	|	loss
0.93	|	0.44
0.93	|	0.42
0.93	|	0.41
0.93	|	0.39


/home/ecowan/IdeaProjects/smartnoise-core-python/venv/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [31]:
model = PumsModule(len(problem['predictors']), 2)

In [32]:
optimizer = torch.optim.SGD(model.parameters(), .1)
for i in range(100):
    loss = model.loss(batch)
    loss.backward()
    optimizer.step()
    print(model.linear1.weight.grad)
    optimizer.zero_grad()

tensor([[2.7798e-01, 2.7981e-01, 6.5369e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.4907e-01, 9.2411e-02, 2.8562e-03],
        [2.4625e+00, 3.7981e+00, 6.2600e-02],
        [2.3981e+00, 2.4020e+00, 5.6004e-02]])
tensor([[ 1.1387e+00,  1.2867e+00,  3.7147e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.4686e-01, -9.1864e-02, -4.5447e-03],
        [-7.0453e-02, -7.8051e-01, -1.2719e-02],
        [-1.0265e-02, -3.3190e-03, -4.3899e-04]])
tensor([[ 0.1626,  0.0895,  0.0019],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0864,  0.0471,  0.0012],
        [-0.0987, -0.4337, -0.0134],
        [ 0.0552,  0.0140,  0.0012]])
tensor([[ 0.1089,  0.0535,  0.0014],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0539,  0.0260,  0.0006],
        [-0.0581, -0.0422, -0.0096],
        [ 0.0443,  0.0106,  0.0010]])
tensor([[ 0.0921,  0.0441,  0.0012],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0396,  0.0179,  0.0004],
        [-0.0535, -0.0165, -0.0092],
   

In [ ]:
gradients['linear1.weight']

In [89]:
def reduce_all_params(grads):
    for x in zip(*map(torch.flatten, grads)):
        yield [y.item() for y in x]

model = PumsModule(len(problem['predictors']), 2)

for epoch in range(0,2):
    param_grads = dict((name, []) for name, _ in model.named_parameters())
    for batch in train_loader:
        loss = model.loss(batch)
        loss.backward()
        for name, param in model.named_parameters():
            param_grads[name].append(param.clone())
        optimizer.zero_grad()
    grads = reduce_all_params(param_grads)
    print(list(grads))
    for name, grad in zip(model.parameters(), grads):
        model.grad = grad
    optimizer.step()
    optimizer.zero_grad()

TypeError: flatten(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
    next(map(torch.flatten, gradients['linear1.weight']))

In [ ]:
torch.narrow(torch.narrow(example_stacked_tensor, 1, 0, 1), 2, 0, 1)

In [41]:
torch.flatten(gradients['linear1.weight'][0])

tensor([-6.9835e+00, -1.6266e+01, -3.2259e-01,  5.7672e+00,  1.2583e+01,
         2.5878e-01, -5.1285e-01, -2.6154e-01, -1.5506e-02,  1.4230e+01,
         3.3144e+01,  6.5731e-01,  2.2056e+00,  1.5239e+00,  6.0899e-02])

In [ ]:
import numpy as np
import opendp.smartnoise.core as sn

with sn.Analysis():
    grads = [x.mean().item() for x in gradients['linear1.weight']]
    dataset = sn.literal(value=grads, value_public=False)
    typed = sn.cast(dataset, atomic_type={
        bool: 'bool', float: 'float', int: 'int', str: 'str'
    }[float], true_label=True, lower=0, upper=10)
    resized = sn.resize(typed, number_columns=1, lower=0., upper=10.)
    data = sn.to_dataframe(resized, names=['linear1.weight'])
    print(data)
    candidates = [0., 3., 4., 7., 10., 12.]

    median_scores = sn.median(
         data,
         candidates=candidates,
         data_rows=len(gradients))
    # print(median_scores.value)
    # dp_median = sn.exponential_mechanism(median_scores, candidates=candidates, privacy_usage={"epsilon": 1.})
    # print(dp_median.value)

In [4]:
for batch in train_loader:
    for x in batch:
        print(x.size())
        break

torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([949, 3])


In [6]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.grad)

linear1.weight tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
linear1.bias tensor([0., 0., 0., 0., 0.])
linear2.weight tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
linear2.bias tensor([0., 0.])


In [7]:
for param in model.parameters():
    print(param.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0., 0., 0., 0.])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([0., 0.])


In [8]:
for x,y in model.state_dict().items():
    print(x)
    print(y)

linear1.weight
tensor([[-0.1316, -0.0025,  0.1985],
        [-1.0574, -2.3805, -0.5121],
        [ 0.2873, -0.5425, -0.2715],
        [-0.2792, -0.0577,  0.0040],
        [-0.1770,  0.2283, -0.0225]])
linear1.bias
tensor([ 0.0158, -0.2395,  0.4055, -0.3556,  0.1591])
linear2.weight
tensor([[ 0.1364, -0.3030, -0.1385,  0.1452, -0.1271],
        [-0.0288,  0.2484,  0.0787, -0.2871,  0.0441]])
linear2.bias
tensor([-0.3465,  0.6554])


{'public': False, 'year': 2010, 'record_type': 'person', 'state': 'ct'}